In [1]:
!git clone https://github.com/afrenkai/DS-3010-Final.git

Cloning into 'DS-3010-Final'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 131 (delta 55), reused 81 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 18.28 MiB | 7.87 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [2]:
%cd DS-3010-Final

/content/DS-3010-Final


In [3]:
!ls
!pip install torcheval

cv.py  LICENSE	   new_3010_proj_work_ben.ipynb  requirements.txt
Data   Main.ipynb  README.md
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 2.8 MB/s eta 0:00:00


In [24]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from lightgbm import LGBMRegressor
from torcheval.metrics import R2Score

In [5]:
train_df = pd.read_csv('Data/SGEMM_train.csv')
val_df = pd.read_csv('Data/SGEMM_val.csv')

#TODO: read test data (already in data dir), see what's going on in lightgbm, get r2 for the neural net

# preprocessing

In [74]:
def norm(x, xmin, xmax, a, b):
  '''
  Restricts x values to range of [xmin, xmax]
  '''
  numerator = x - xmin
  denominator = xmax - xmin
  return (numerator / denominator) * (b - a) + a

In [7]:
cols_to_combine = ['Run1 (ms)', 'Run2 (ms)', 'Run3 (ms)', 'Run4 (ms)']

In [75]:
def preprocess(df: pd.DataFrame):
  df['DELTA_RUNTIME'] = df.apply(
      lambda row: np.mean([row['Run1 (ms)'], row['Run2 (ms)'], row['Run3 (ms)'], row['Run4 (ms)']]),
      axis=1
  )
  for col in df.columns:
    if col in cols_to_combine:
      df = df.drop(col, axis = 1) #removes redundant cols


  min = 0
  max = 1

  df = df.apply(
      lambda row: (norm(row, row.min(), row.max(), min, max))
  )
  x = df.iloc[:, :14] # features
  y = df.iloc[:, -1:] # target
  return x, y


# LightGBM

In [9]:

x_tr, y_tr = preprocess(train_df)

train_data = lgb.Dataset(x_tr, label=y_tr)
x_val, y_val = preprocess(val_df)
# Create a LightGBM dataset for testing with features X_val and labels Y_val,
# and specify the reference dataset as train_data for consistent evaluation
val_data = lgb.Dataset(x_val, label=y_val, reference=train_data)
params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[
                val_data])


# Create an instance of the LightGBM Regressor with the RMSE metric.
model = LGBMRegressor(metric='mse')

# Train the model using the training data.
model.fit(x_tr, y_tr)

y_train = model.predict(x_tr)
y_v = model.predict(x_val)
print("Training MSE:", mse(y_tr, y_train))
print("Validation MSE:", mse(y_val, y_v))

print('train r2:', r2_score(y_tr, y_train))
print('val r2:', r2_score(y_val, y_v))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017275 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 193280, number of used features: 14
[LightGBM] [Info] Start training from score 0.061354
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030284 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 40
[LightGBM] [Info] Number of data points in the train set: 193280, number of used features: 14
[LightGBM] [Info] Start training from score 0.061354
Training MSE: 0.00012048252219381041
Validation MSE: 0.0001191615359407208
train r2: 0.9901529624802171
val r2: 0.9903041142135522


# Simple Neural Net

In [10]:
class GPUNN(nn.Module):
  def __init__(self, in_feat, out_feat):
    super(GPUNN, self).__init__()
    self.device = 'cuda' if torch.cuda.is_available else 'cpu'
    self.L1 = nn.Linear(in_feat, 64, device=self.device)
    self.L2 = nn.Linear(64, out_feat, device = self.device)
    self.relu = nn.ReLU()
    self.bn1 = nn.BatchNorm1d(64, device = self.device)
    self.bn2 = nn.BatchNorm1d(out_feat, device = self.device)

  def forward(self, x):
    x = self.bn1(self.relu(self.L1(x)))
    x = self.bn2(self.L2(x))
    return x



In [76]:
# training output example was basen on an example from pytorch on using CNNs for ImageNet. https://github.com/pytorch/examples/blob/main/mnist/main.py
# lines 45-48 in original code, lines roughly 13-16 in ours
def train(model: nn.Module, train_dl: DataLoader, batch_size, device, n_epochs, optimizer, criterion):
    model.train()
    for epoch in trange(n_epochs):
        for batch, (data, target) in enumerate(train_dl):
            data, target = data.to(device).float(), target.to(device).float()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            if batch % 1000 == 0:
                print('\nTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch + 1, batch * len(data), len(train_dl.dataset),
                    100. * batch / len(train_dl), loss.item()))

    torch.save(model.state_dict(), 'nn.pth')
    print('model saved to nn.pth')


In [67]:
#r2 score stuff taken from https://pytorch.org/torcheval/main/generated/torcheval.metrics.R2Score.html
#I think meaning loss over batches is fair, since that's a relatively good indicator of performance.
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    losses = []
    metric = R2Score()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device).float(), target.to(device).float()
            output = model(data)

            test_loss = criterion(output, target)
            # print('loss bien')
            # print(f'Target Tensor: {target.detach().cpu().numpy()}\n, Output Tensor:{output.detach().cpu().numpy()}\n')

            metric.update(output, target)
            # print(f' r2 rn: {metric.compute()}')

            # print('r2 bien')
            losses.append(test_loss)


    # print(test_loss / len(test_loader.dataset))
    # print(np.mean(r2s))
    r2 = metric.compute()
    r2 = float(r2.detach().cpu().numpy())
    # print(type(r2))
    # print(round(r2, 4))
    return (np.mean([ten.detach().cpu().numpy() for ten in losses]), round(r2, 4) )






In [68]:
def create_dls(x: pd.DataFrame, y:pd.DataFrame, split: str = 'train'):
  x = x.loc[:, :].values #conv to np array
  y = y.loc[:, :].values
  x_ten = torch.tensor(x) # conv to tensor
  y_ten = torch.tensor(y)
  ds = TensorDataset(x_ten, y_ten)
  if split == 'train':
    dl = DataLoader(ds, batch_size = 32, shuffle = True) #safe to shuffle since its train
  else:
    dl = DataLoader(ds, batch_size = 32, shuffle = False) #assuming test/val, can't shuffle
  return ds, dl

In [69]:
x_tr, y_tr = preprocess(train_df)
model = GPUNN(len(x_tr.columns), len(y_tr.columns))
print(model)
_, train_dl = create_dls(x_tr, y_tr)
criterion = nn.MSELoss() # MSE, like we've been using everywhere else
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-4) # pretty standard, no need for k-fold since the method works pretty well without tuning
train(model, train_dl, 32, model.device, 10, optimizer, criterion) # 10 epochs to get a baseline.

GPUNN(
  (L1): Linear(in_features=14, out_features=64, bias=True)
  (L2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


  0%|          | 0/10 [00:00<?, ?it/s]


Train Epoch: 1 [0/193280 (0%)]	Loss: 1.020140

Train Epoch: 1 [32000/193280 (17%)]	Loss: 0.653175

Train Epoch: 1 [64000/193280 (33%)]	Loss: 0.587248

Train Epoch: 1 [96000/193280 (50%)]	Loss: 0.366149

Train Epoch: 1 [128000/193280 (66%)]	Loss: 0.320002

Train Epoch: 1 [160000/193280 (83%)]	Loss: 0.241567


 10%|█         | 1/10 [00:11<01:44, 11.57s/it]


Train Epoch: 1 [192000/193280 (99%)]	Loss: 0.087700

Train Epoch: 2 [0/193280 (0%)]	Loss: 0.171225

Train Epoch: 2 [32000/193280 (17%)]	Loss: 0.042149

Train Epoch: 2 [64000/193280 (33%)]	Loss: 0.049729

Train Epoch: 2 [96000/193280 (50%)]	Loss: 0.018328

Train Epoch: 2 [128000/193280 (66%)]	Loss: 0.019041

Train Epoch: 2 [160000/193280 (83%)]	Loss: 0.008792


...


Train Epoch: 8 [192000/193280 (99%)]	Loss: 0.006268

Train Epoch: 9 [0/193280 (0%)]	Loss: 0.003335

Train Epoch: 9 [32000/193280 (17%)]	Loss: 0.001990

Train Epoch: 9 [64000/193280 (33%)]	Loss: 0.002606

Train Epoch: 9 [96000/193280 (50%)]	Loss: 0.001139

Train Epoch: 9 [128000/193280 (66%)]	Loss: 0.001912

Train Epoch: 9 [160000/193280 (83%)]	Loss: 0.002447

Train Epoch: 9 [192000/193280 (99%)]	Loss: 0.001051


 90%|█████████ | 9/10 [01:49<00:12, 12.18s/it]


Train Epoch: 10 [0/193280 (0%)]	Loss: 0.005938

Train Epoch: 10 [32000/193280 (17%)]	Loss: 0.001765

Train Epoch: 10 [64000/193280 (33%)]	Loss: 0.002090

Train Epoch: 10 [96000/193280 (50%)]	Loss: 0.002031

Train Epoch: 10 [128000/193280 (66%)]	Loss: 0.002985

Train Epoch: 10 [160000/193280 (83%)]	Loss: 0.006218


100%|██████████| 10/10 [02:02<00:00, 12.23s/it]


Train Epoch: 10 [192000/193280 (99%)]	Loss: 0.001298
model saved to nn.pth


In [72]:
x_val, y_val = preprocess(val_df)
_, val_dl = create_dls(x_val, y_val)
criterion = nn.MSELoss()
val_loss, val_r2= test(model, model.device, val_dl, criterion)
print(f'Neural Network Validation Mean Squared Error: {val_loss:.6f}')
print(f'Neural Network Validation R2: {val_r2}')

Neural Network Validation Mean Squared Error: 0.000864
Neural Network Validation R2: 0.9297


In [73]:
# model.load_state_dict(torch.load('nn.pth', weights_only=True))
# print("Model's state_dict:")
# model.state_dict()

Model's state_dict:


OrderedDict([('L1.weight',
              tensor([[-1.7224e-02,  1.5817e-01,  6.4280e-02, -4.4834e-01, -5.1279e-01,
                       -2.1299e-03, -3.4860e-03,  5.8696e-02, -1.0740e-03, -1.0833e-03,
                       -1.0295e-03, -3.3369e-03,  6.1607e-02,  8.3909e-02],
                      [-8.2129e-02,  1.8607e-01, -5.4309e-02, -1.2423e-01, -1.9301e-01,
                        1.2961e-02, -7.2939e-02,  1.2869e-01,  2.1684e-01,  5.6681e-02,
                        2.2671e-01, -1.1918e-01, -4.6155e-02, -1.6433e-01],
                      [ 1.1286e-01,  9.7598e-02,  4.3918e-02, -1.7781e-01, -1.8503e-01,
                        1.7688e-03, -9.8405e-04,  3.0680e-02,  2.4066e-04,  1.6771e-03,
                       -6.6359e-04,  8.2731e-04,  4.4551e-02,  4.4426e-02],
                      [ 1.8135e-01, -1.2530e-01,  9.1653e-02, -2.0359e-02,  1.0247e-01,
                        2.5196e-01, -5.1992e-02,  1.1030e-01,  8.4600e-02, -1.4288e-01,
                        1.0022e-01, -1.75

PCA, Lasso & Linear Regression - Katelyn

In [48]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

X_train = pca.fit_transform(x_tr)
X_test = pca.transform(x_val)

explained_variance = pca.explained_variance_ratio_

print(explained_variance)


[0.10205183 0.10191195]


In [18]:
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

#From Sklearn --> Lasso documentation

#Setting alpha
lasso = Lasso(alpha=0.00001)

#Fitting to training data
lasso.fit(X_train, y_train)

#making y predictions based on the X_test, given by the PCA
y_pred = lasso.predict(X_test)

#Getting the MSE
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")

print("Coefficients:", lasso.coef_)
#very low lasso coefficents

Mean Squared Error: 0.012236981126572908
Coefficients: [ 0.01265718 -0.00163292]


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

# Initialize LDA and fit the model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Predict on the test set
y_pred = linreg.predict(X_test)

# Evaluate accuracy
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")

#This does very well, but the other methods are preferable for exploring.
print('train r2:', r2_score(y_tr, y_train))

Mean Squared Error: 0.012236947492410348
train r2: 0.9901529624802171
